In [1]:
hex_to_str = {0: 'f', 1: 'g', 2: 'h', 3: 'i', 4: 'j', 5: 'k', 6: 'l', 7: 'm', 8: 'n', 9: 'o', 10: 'p', 11: 'q', 12: 'r', 13: 's', 14: 't', 15: 'u'}

#number-->cipher_hex
def byte_hex(b):
    return hex_to_str[b//16]+ hex_to_str[b%16]

#2 cipher text-->ASCII
def map_to_ASCII(st):
    char = chr(16*(ord(st[0]) - ord('f')) + ord(st[1]) - ord('f'))
    return char

#8 byte or 16 char cipher text-->ASCII
def block_to_ASCII(c):
    plain = ""
    for i in range(0, len(c), 2):
        plain += map_to_ASCII(c[i:i+2])
    return plain

In [2]:
from pyfinite import ffield

exp_store = [[-1]*128 for i in range(128)]

F = ffield.FField(7)

def Expo (no, pow_):
    if exp_store[no][pow_] != -1:
        return exp_store[no][pow_]

    result = 0;
    if pow_ == 0:
        result = 1
    elif pow_ == 1:
        result = no
    elif pow_%2 == 0:
        sqrt_no = Expo(no, pow_>>1)
        result = F.Multiply(sqrt_no, sqrt_no)
    else:
        sqrt_no = Expo(no, pow_>>1)
        result = F.Multiply(sqrt_no, sqrt_no)
        result = F.Multiply(no, result)

    exp_store[no][pow_] = result
    return result

def addVectors (v1, v2):
    result = [0]*8
    for i, (e1, e2) in enumerate(zip(v1, v2)):
        result[i] = e1 ^ e2
    return result

def scalarMultiplication (v, elem):
    result = [0]*8
    for i, e in enumerate(v):
        result[i] = F.Multiply(e, elem)
    return result

def LinearTransformation (mat, elist):
    result = [0]*8
    for row, elem in zip(mat, elist):
        result = addVectors(scalarMultiplication(row, elem), result)
    return result

#This function mimics EAEAE
def EAEAE (plaintext, lin_mat, exp_mat):
    plaintext = [ord(c) for c in plaintext]
    Output = [[0 for j in range (8)] for i in range(8)]
    for ind, elem in enumerate(plaintext):
        Output[0][ind] = Expo(elem, exp_mat[ind])

    Output[1] = LinearTransformation(lin_mat, Output[0])

    for ind, elem in enumerate(Output[1]):
        Output[2][ind] = Expo(elem, exp_mat[ind])

    Output[3] = LinearTransformation(lin_mat, Output[2])

    for ind, elem in enumerate(Output[3]):
        Output[4][ind] = Expo(elem, exp_mat[ind])
    return Output[4]

In [3]:
#This list will consist of all possible exponents
poss_exp = [[] for i in range(8)]
#This list will consist of all possible diagonal values
poss_aii = [[[] for i in range(8)] for j in range(8)]
input_file = open("inputs.txt", 'r')
output_file = open("outputs.txt", 'r')
for idx, (in_line, out_line) in enumerate(zip(input_file.readlines(), output_file.readlines())):
    inp_str = []
    out_str = []
    #Converting each input to corresponding hex values
    for hex_in in in_line.strip().split(" "):
        inp_str.append(block_to_ASCII(hex_in)[idx])
    for hex_out in out_line.strip().split(" "):
        out_str.append(block_to_ASCII(hex_out)[idx])
        
    for i in range(1, 127):
        for j in range(1, 128):
            flag = True
            for inps, outs in zip(inp_str, out_str):
                #We iterate over all possible values of ei and ajj to check if input maps to output
                if ord(outs) != Expo(F.Multiply(Expo(F.Multiply(Expo(ord(inps), i), j), i), j), i):
                    flag = False
                    break
            if flag:
                #If yes, then we append them to corresponding possible lists
                poss_exp[idx].append(i)
                poss_aii[idx][idx].append(j)
print(poss_exp)
poss_aii

[[20, 108], [53, 83, 118], [40, 89, 125], [11, 34, 82], [2, 38, 87], [29, 43, 55], [22, 37, 68], [1, 19, 107]]


[[[84, 67], [], [], [], [], [], [], []],
 [[], [72, 101, 70], [], [], [], [], [], []],
 [[], [], [43, 14, 72], [], [], [], [], []],
 [[], [], [], [6, 9, 12], [], [], [], []],
 [[], [], [], [], [103, 8, 112], [], [], []],
 [[], [], [], [], [], [17, 110, 11], [], []],
 [[], [], [], [], [], [], [27, 66, 70], []],
 [[], [], [], [], [], [], [], [104, 38, 38]]]

In [4]:
input_file = open("inputs.txt", 'r')
output_file = open("outputs.txt", 'r')
for idx, (in_line, out_line) in enumerate(zip(input_file.readlines(), output_file.readlines())):
    #Considering only first 6 pairs
    if idx > 6 :
        break
    inp_str = []
    out_str = []
    #Converting each input to corresponding hex values
    for hex_in in in_line.strip().split(" "):
        inp_str.append(block_to_ASCII(hex_in)[idx]) 
    for hex_o in out_line.strip().split(" "):
        out_str.append(block_to_ASCII(hex_o)[idx+1])
    for i in range(1, 128):
        #We iterate over all possible pairs of exponents and diagonal values
        for p1, e1 in zip(poss_exp[idx+1], poss_aii[idx+1][idx+1]):
            for p2, e2 in zip(poss_exp[idx], poss_aii[idx][idx]):
                flag = True
                for inp, outp in zip(inp_str, out_str):
                    #We substitute the pairs ad=nd iterate over all possible values of i
                    #We do this by forming virtual triangle (aii,aij,ajj)
                    if ord(outp) != Expo((F.Multiply(Expo(F.Multiply(Expo(ord(inp), p2), e2), p2), i) ^ F.Multiply(Expo(F.Multiply(Expo(ord(inp), p2), i), p1), e1)), p1):
                        flag = False
                        break
                if flag:
                    #If we find such value, then we can discard other possibilities and finalize the lists
                    poss_exp[idx+1] = [p1]
                    poss_aii[idx+1][idx+1] = [e1]
                    poss_exp[idx] = [p2]
                    poss_aii[idx][idx] = [e2]
                    poss_aii[idx][idx+1] = [i]
print(poss_exp)
poss_aii

[[20], [118], [40], [82], [87], [55], [22], [19]]


[[[84], [114], [], [], [], [], [], []],
 [[], [70], [28], [], [], [], [], []],
 [[], [], [43], [31], [], [], [], []],
 [[], [], [], [12], [108], [], [], []],
 [[], [], [], [], [112], [110], [], []],
 [[], [], [], [], [], [11], [83], []],
 [[], [], [], [], [], [], [27], [4]],
 [[], [], [], [], [], [], [], [38]]]

In [5]:
for index in range(6):
    #As we have already found element next to diagonal thus skipping two elements every time
    of = index + 2
    
    exp_list = [e[0] for e in poss_exp]
    lin_trans_list = [[0 for i in range(8)] for j in range(8)]
    #We fill all the empty [] elements with 0
    for i in range(8):
        for j in range(8):
            lin_trans_list[i][j] = 0 if len(poss_aii[i][j]) == 0 else poss_aii[i][j][0]
    inp = open("inputs.txt", 'r')
    out = open("outputs.txt", 'r')
    for ind, (iline, oline) in enumerate(zip(inp.readlines(), out.readlines())):
        if ind > (7-of):
            continue
        inpString = [block_to_ASCII(msg) for msg in iline.strip().split(" ")]
        outString = [block_to_ASCII(msg) for msg in oline.strip().split(" ")]
        #We iterate over all possible values of ai,j to find which one satisfies EAEAE = Output
        for i in range(1, 128):
            lin_trans_list[ind][ind+of] = i
            flag = True
            for inps, outs in zip(inpString, outString):
                if EAEAE(inps, lin_trans_list, exp_list)[ind+of] != ord(outs[ind+of]):
                    flag = False
                    break
            if flag:
                poss_aii[ind][ind+of] = [i]
    inp.close();
    out.close();
#We fill all the empty [] elements with 0
lin_trans_list = [[0 for i in range(8)] for j in range(8)]
for i in range(8):
    for j in range(8):
        lin_trans_list[i][j] = 0 if len(poss_aii[i][j]) == 0 else poss_aii[i][j][0]

print(exp_list)
lin_trans_list

[20, 118, 40, 82, 87, 55, 22, 19]


[[84, 114, 14, 123, 97, 31, 17, 95],
 [0, 70, 28, 27, 39, 40, 121, 13],
 [0, 0, 43, 31, 3, 29, 20, 83],
 [0, 0, 0, 12, 108, 54, 101, 25],
 [0, 0, 0, 0, 112, 110, 4, 14],
 [0, 0, 0, 0, 0, 11, 83, 68],
 [0, 0, 0, 0, 0, 0, 27, 4],
 [0, 0, 0, 0, 0, 0, 0, 38]]

In [6]:
#Final Matrices as we found them in above step
LINEAR_TRANS = [[84, 114, 14, 123, 97, 31, 17, 95], 
                 [0, 70, 28, 27, 39, 40, 121, 13], 
                 [0, 0, 43, 31, 3, 29, 20, 83], 
                 [0, 0, 0, 12, 108, 54, 101, 25], 
                 [0, 0, 0, 0, 112, 110, 4, 14], 
                 [0, 0, 0, 0, 0, 11, 83, 68], 
                 [0, 0, 0, 0, 0, 0, 27, 4], 
                 [0, 0, 0, 0, 0, 0, 0, 38]]
EXPONENT = [20, 118, 40, 82, 87, 55, 22, 19]

In [7]:
#Two halves of password
password1 = 'lhiqigjniogigijt'
password2 = 'kfinfrjuhuffgrmq'

#We iterate over all possible 128 byte values and perform EAEAE to check for which input the password (half) matches
def DecryptPassword(password):
    pwd = block_to_ASCII(password)
    op = ""
    for idx in range(8):
        for ans in range(128):
            inp = op + byte_hex(ans)+(16-len(op)-2)*'f'
            if ord(pwd[idx]) == EAEAE(block_to_ASCII(inp), LINEAR_TRANS, EXPONENT)[idx]:
                op += byte_hex(ans)
                break
    return op

print(block_to_ASCII(DecryptPassword(password1))+block_to_ASCII(DecryptPassword(password2)))

twqxybopdk000000
